## Use tensorflow model to generate graffiti detections
This example demonstrates how to use a **frozen tensorflow model** to generate detections for a set of images.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from utils import data_sequence, classification_auxiliary
import generateAnnotationFromModelsPredictions, eval_model

Load the detection graph from an exported checkpoint file:

In [ ]:
model_path = 'models/tensorflow_models/ssd_inception_v2_training_2/ckpt-64000/frozen_inference_graph.pb'
detection_graph = generateAnnotationFromModelsPredictions.load_tf_graph(model_path)

It is important to pass **keras_preprocessing_mode=False** parameter to the **Prediction_Data_Sequence** call when using tensorflow graphs.
The path_to_test_images contains the images. It is recommended that this directory should contain images only.
The Prediction_Data_Sequence produces batches of numpy image arrays and file paths where the predictions can be stored.

In [ ]:
path_to_test_images = 'data/images/detection_test'
image_size = 460
data_limit_of_n_images = 10
batch_size = 4
keras_preprocessing_mode = False

In [ ]:
data_generator = data_sequence.Prediction_Data_Sequence(
    path_to_test_images,
    keras_preprocessing_mode=keras_preprocessing_mode,
    max_data_size=data_limit_of_n_images,
    image_size=image_size,
    grid_sizes=[0,0],
    batch_size=batch_size,
)

Use detection graph to run inference for each batch. The final detections are generated from the predictions according to the specified parameters.

In [ ]:
maximum_overlap_between_distinct_detections = 0.4
only_consider_objects_with_higher_confidence_than = 0.3
detect_at_most_n_distinct_objects = 10

In [ ]:
for h in range(len(data_generator)):
    x_batch,y_batch = data_generator[h]
    batch_predictions = eval_model.generate_tf_prediction_batch(x_batch,detection_graph)
    for i in range(batch_predictions.shape[0]):
        predictions = batch_predictions[i]
        positives,_ = eval_model.detect(
            predictions,
            iou_overlap_threshold=maximum_overlap_between_distinct_detections,
            confidence_threshold=only_consider_objects_with_higher_confidence_than,
            max_number_of_detections=detect_at_most_n_distinct_objects,
        )
        
        pred_image_np = x_batch[i]
        eval_model.plot_to_img(pred_image_np,positives,predictions,'red',12,True)
        eval_model.plot_to_img(pred_image_np,positives,predictions,'white',6,True)
        plt.imshow(pred_image_np)
        plt.show()
        